# Automatic Speech Recognition (ASR) Tutorial 

## Fine-tune a pretrained, multilingual ASR model on FLEURS

In this tutorial, we will be evaluating and improving a multilingual ASR model for a language in the FLEURS dataset. We will focus on **Hausa**, but you can follow along in any language in FLEURS. See the FLEURS dataset paper for a list of supported languages: https://arxiv.org/abs/2205.12446

## Setting up your coding environment 

You can run follow along and run the lines of code in this notebook, and also utilize the scripts found in this GitHub respository. You will need to create a virtual environment for this project so you can download all the required packages without affecting your other projects. We have provided an `environment.yml` file that you can use to create a conda virtual environment :